# RMSPE manipulation for debugging purpose 

A few of my recent submissions have scores that are > 10. I'd like to figure out if it's caused by my wrong assumptions of the data (particularly for pairs of (stock_id, time_id)).

In this notebook, we derive a formula to manipulate the RMSPE score, so we can get some metadata of the test set. The formula can display any number that are >= 1.

> !!! PLEASE NOTE: The intention of this notebook is not cheating, violating the rule or doing anything weried!

> !! If you get any result by using this notebook, I kindly encourage you to share the result with everyone :)

Strikethrough: ~~!! This work is still in progress, because I haven't found a non-filler target value in the test set :(~~

**Update 28 Jul 2021:** I found that the 5th time_id for stock 0 is a non-filler data.

# Formula Derivation

Notations:
- $\hat{y}$ - the vector of predictions
- $y$ - the vector of targets
- $n$ - the length of target (or predictions)

Define the function $eval(\hat{y})$ such that
$$ RMSPE = eval(\hat{y}) := \sqrt{ \frac{1}{n}\sum_{i=1}^{n} ((y_i - \hat{y}_i)/y_i)^2 } $$

Our goal is to derive a $\mathbb{R}^n$-valued function $f(x)$ that satisfies
$$ RMSPE = eval(f(RMSPE)) $$

If $\hat{y}=(0,\ldots,0)$, then $RMSPE=1.0$. By using this fact, if we know the value of non-filler target $y_k$ for some $k$, we can manipulate the $RMSPE$ via $\hat{y}_k$.

Now, let's assume that
- $y_0$ is a non-filler target
- $y_0$ is known,
- $\hat{y}_0 > y_0$,
- $\hat{y}_i = 0$ for all $i>0$.

We can derive that
$$\begin{align}
\sum_{i=1}^n((y_i-\hat{y}_i)/y_i)^2 &= n\times RMSPE^2 \\
(\hat{y}_0/y_0 - 1)^2 + (n-1) &= n\times RMSPE^2 \\
\hat{y}_0 &= y_0 \left( \sqrt{n\times RMSPE^2 - (n-1) } + 1\right) \\
\end{align}$$

Define the scaler-valued function $h(x)$ and the $\mathbb{R}^n$-valued function $f(x)$ as follows:
$$\begin{align}
h(x) &:= y_0 \left( \sqrt{n\times x^2 - (n-1) } + 1\right) \\
f(x) &:= (h(x), 0, 0, \ldots, 0)
\end{align}$$
We have 
$$ RMSPE = eval(f(RMSPE)) $$
Q.E.D.

Based on the above results, let's define the following functions.

In [ ]:
def rmspe_decoder(rmspe, y_hat_k, n):
    """ Calculate y_k with the displayed rmspe and the manually set y_hat_k. """
    return y_hat_k / (np.sqrt(n * rmspe * rmspe - (n-1)) + 1)

def rmspe_encoder(rmspe, y_k, n):
    """ Calculate the y_hat_k that displays given rmspe on the LB. """
    return y_k * (np.sqrt(n * rmspe * rmspe - (n - 1)) + 1)

# Peek the value a non-filler target $y_k$

To make it work, we need to spend a few submissions to peek the value of a target $y_k$ which is a filler data. 

However, I have spent a few submission but haven't find such $k$ yet, because there are too many filler data in the test set. 

One way to do it is to use binary search, which will cost about $\lceil log_2(150000)\rceil = 18$ submissions. This is too expensive!

I will continue work on this notebook if such $y_k$ is found.

**Update:** I've tried to iterate the time_id for stock 0, and luckly found one instance.

By setting the prediction of the 4th time_id for stock 0 to 1, I got public score = 2.04407.

# Encode the score you want to display and make a submission

In [ ]:
import glob
import pandas as pd
import numpy as np

DEBUG = 0

In [ ]:
def load_book_data_by_id(stock_id):
    """ Load book data by stock_id. """
    train_test = 'train' if DEBUG else 'test'
    df = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_{train_test}.parquet/stock_id={stock_id}')
    return df

def make_submission():
    """ Make a submission whose target values are all zeros. """
    train_test = 'train' if DEBUG else 'test'
    list_stock_id = sorted([int(path.split('=')[-1]) for path in 
        glob.glob(f'../input/optiver-realized-volatility-prediction/book_{train_test}.parquet/*')
    ])
    list_df = []
    for stock_id in list_stock_id:
        # loading data
        df_book = load_book_data_by_id(stock_id)
        # make submission for one stock
        df_ = pd.DataFrame(df_book['time_id'].unique(), columns=['time_id'])
        df_['time_id'] = [f'{stock_id}-{time_id}' for time_id in df_['time_id']]
        list_df.append(df_)
        if DEBUG: break
    # Make submission
    df_submission = pd.concat(list_df).rename(columns={'time_id': 'row_id'})
    df_submission['target'] = 0
    return df_submission

def inject_rmspe_code_to_submission(code, df_submission):
    """ Change the target of the special row_id to [code]. """
    list_row_id_of_stock_0 = df_submission.loc[df_submission.row_id.str.startswith('0-'), 'row_id'].tolist()
    list_time_id_of_stock_0 = sorted([int(row_id.split('-')[-1]) for row_id in list_row_id_of_stock_0])
    try:
        special_time_id = list_time_id_of_stock_0[4]
        df_submission.loc[df_submission.row_id==f'0-{special_time_id}', 'target'] = code
    except:
        None
    return df_submission

In [ ]:
def make_manipulated_submission(manipulated_rmspe):
    """Combine the above functions into one."""
    # Constant
    PUBLIC_RMSPE = 2.04407
    Y_HAT_K = 1
    # make a default submission
    df_submission = make_submission()
    n = len(df_submission)
    # encoding
    y_k = rmspe_decoder(PUBLIC_RMSPE, Y_HAT_K, n)
    rmspe_code = rmspe_encoder(manipulated_rmspe, y_k, n)
    # make submission
    df_submission = inject_rmspe_code_to_submission(rmspe_code, df_submission)
    return df_submission

# Let's see the magic! Show 42?

In [ ]:
manipulated_rmspe = 42
df_submission = make_manipulated_submission(manipulated_rmspe)
df_submission
df_submission.to_csv('submission.csv', index=False)
df_submission.head()